<a href="https://colab.research.google.com/github/Areliortiz/trabajo/blob/main/ambulancias_optimizar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 78.6 MB/s eta 0:00:00


In [5]:
from pulp import LpProblem, LpMinimize, LpMaximize, LpVariable, lpSum, LpBinary, value

# Datos iniciales
numberofSupply = 4  # Número de estaciones candidatas, hospitales
numberofDemand = 16  # Número de puntos de demanda, ubicaciones
D_max = 15  # Distancia máxima (puede ajustarse según el caso)

# Matriz de cobertura (1 si dentro del rango D_max, 0 si no)
N = [
    [1, 0, 1, 0],  # Punto de demanda 1 cubierto por estaciones 1 y 3 gus
    [1, 1, 0, 1],  # Punto de demanda 2 cubierto por estaciones 1 y 2 iz
    [0, 1, 1, 0],  # Punto de demanda 3 cubierto por estaciones 2 y 3 ve
    [1, 0, 0, 1]   # Punto de demanda 4 solo cubierto por estación 1 mi
]

# Pesos asociados a cada punto de demanda (para MCLP)
weights = [90, 100, 80, 60]

# Número máximo de estaciones disponibles (para MCLP)
max_stations = 4

# Crear modelo para LSCM
model_lscm = LpProblem("Location_Set_Covering_Model", LpMinimize)

# Variables de decisión
x = LpVariable.dicts("x", range(numberofSupply), cat=LpBinary)  # Estaciones
y = LpVariable.dicts("y", range(numberofDemand), cat=LpBinary)  # Puntos cubiertos (para MCLP)

# Función objetivo para LSCM (minimizar número de estaciones)
model_lscm += lpSum(x[j] for j in range(numberofSupply))

# Restricciones para asegurar cobertura en LSCM
for i in range(numberofDemand):
    model_lscm += lpSum(N[i][j] * x[j] for j in range(numberofSupply)) >= 1

# Resolver LSCM
model_lscm.solve()

# Imprimir resultados para LSCM
print("\n--- Location Set Covering Model (LSCM) ---")
print("Estado del modelo:", model_lscm.status)
print("Número mínimo de estaciones:", value(model_lscm.objective))
for j in range(numberofSupply):
    print(f"Estación {j + 1} activa:", x[j].value())

# Crear modelo para MCLP
model_mclp = LpProblem("Maximal_Covering_Location_Problem", LpMaximize)

# Función objetivo para MCLP (maximizar cobertura de demanda)
model_mclp += lpSum(weights[i] * y[i] for i in range(numberofDemand))

# Restricciones para cobertura en MCLP
for i in range(numberofDemand):
    model_mclp += y[i] <= lpSum(N[i][j] * x[j] for j in range(numberofSupply))

# Restricción en el número de estaciones disponibles
model_mclp += lpSum(x[j] for j in range(numberofSupply)) <= max_stations

# Resolver MCLP
model_mclp.solve()

# Imprimir resultados para MCLP
print("\n--- Maximal Covering Location Problem (MCLP) ---")
print("Estado del modelo:", model_mclp.status)
print("Demanda total cubierta:", value(model_mclp.objective))
for j in range(numberofSupply):
    print(f"Estación {j + 1} activa:", x[j].value())
for i in range(numberofDemand):
    print(f"Punto de demanda {i + 1} cubierto:", y[i].value())


IndexError: list index out of range

In [6]:

from pulp import LpProblem, LpMinimize, LpVariable, lpSum

# Datos iniciales
numberofSupply = 3  # Número de estaciones candidatas, es decir numero de dependencias
numberofDemand = 16  # Número de puntos de demanda,numero de alcaldias
D_max = 15  # Distancia máxima (puede ajustarse según el caso)

# Matriz de cobertura (1 si dentro del rango D_max, 0 si no)
N = [
    [1, 0, 0],  # Punto de demanda 1
    [1, 0, 0],  # Punto de demanda 2
    [1, 0, 0],  # Punto de demanda 3
    [1, 0, 0],  # Punto de demanda 4
    [1, 0, 0],  # Punto de demanda 5
    [1, 0, 0],  # Punto de demanda 6
    [1, 0, 0],  # Punto de demanda 7
    [1, 0, 0],  # Punto de demanda 8
    [1, 0, 0],  # Punto de demanda 9
    [1, 0, 0],  # Punto de demanda 10
    [1, 0, 0],  # Punto de demanda 11
    [0, 1, 0],  # Punto de demanda 12
    [0, 1, 0],  # Punto de demanda 13
    [0, 1, 0],  # Punto de demanda 14
    [0, 1, 0],  # Punto de demanda 15
    [0, 0, 1],  # Punto de demanda 16

]

# Crear un modelo de optimización
model = LpProblem("Minimize_Number_of_Stations", LpMinimize)

# Variables de decisión: x[j] = 1 si la estación j está activa, 0 si no
x = [LpVariable(f"x_{j}", cat="Binary") for j in range(numberofSupply)]

# Función objetivo: minimizar el número de estaciones abiertas
model += lpSum(x[j] for j in range(numberofSupply)), "Objective"

# Restricciones: cada punto de demanda debe ser cubierto al menos por una estación activa
for i in range(numberofDemand):
    model += lpSum(N[i][j] * x[j] for j in range(numberofSupply)) >= 1, f"Demand_Coverage_{i+1}"

# Resolver el problema
model.solve()

# Mostrar resultados
print("Estado de la solución:", model.status)
print("Estaciones activadas:")
for j in range(numberofSupply):
    print(f"Estación {j+1}: {'Activa' if x[j].varValue == 1 else 'Inactiva'}")

# Mostrar función objetivo
print("Número mínimo de estaciones abiertas:", sum(x[j].varValue for j in range(numberofSupply)))



Estado de la solución: 1
Estaciones activadas:
Estación 1: Activa
Estación 2: Activa
Estación 3: Activa
Número mínimo de estaciones abiertas: 3.0


In [9]:
from pulp import LpProblem, LpMaximize, LpVariable, lpSum

# Datos iniciales
alcaldias = [
    "Gustavo A Madero", "Azcapotzalco", "Cuauhtémoc", "Cuajimalpa", "Álvaro Obregón",
    "Coyoacán", "Magdalena Contreras", "Tlalpan", "Xochimilco", "Milpa Alta",
    "Tláhuac", "Iztacalco", "Iztapalapa", "Venustiano Carranza", "Benito Juárez", "Miguel Hidalgo"
]
dependencias = {
    "ERUM": ["Gustavo A Madero", "Azcapotzalco", "Cuauhtémoc", "Cuajimalpa", "Álvaro Obregón",
             "Coyoacán", "Magdalena Contreras", "Tlalpan", "Xochimilco", "Milpa Alta"],
    "CRUM": ["Tláhuac", "Iztacalco", "Iztapalapa", "Venustiano Carranza", "Benito Juárez"],
    "Cruz Roja": ["Miguel Hidalgo"]
}
ambulancias_disponibles = {
    "ERUM": 25,
    "CRUM": 8,
    "Cruz Roja": 4
}
demanda_min = 1  # Mínimo de ambulancias por alcaldía
demanda_max = 2  # Máximo de ambulancias por alcaldía

# Crear modelo de optimización
model = LpProblem("Asignación_Ambulancias", LpMaximize)

# Variables de decisión: y[i] = número de ambulancias asignadas a cada alcaldía
y = {alcaldia: LpVariable(f"y_{alcaldia.replace(' ', '_')}", lowBound=demanda_min, upBound=demanda_max, cat="Integer")
     for alcaldia in alcaldias}

# Función objetivo: maximizar la asignación de ambulancias
model += lpSum(y[alcaldia] for alcaldia in alcaldias), "Maximizar_Demanda_Cubierta"

# Restricciones por dependencia
for dependencia, lista_alcaldias in dependencias.items():
    model += lpSum(y[alcaldia] for alcaldia in lista_alcaldias) <= ambulancias_disponibles[dependencia], f"Capacidad_{dependencia}"

# Resolver el modelo
model.solve()

# Mostrar resultados
print("Estado de la solución:", model.status)
print("Asignación de ambulancias por alcaldía:")
for alcaldia in alcaldias:
    print(f"{alcaldia}: {y[alcaldia].varValue} ambulancias")

# Mostrar ambulancias asignadas por dependencia
for dependencia, lista_alcaldias in dependencias.items():
    asignadas = sum(y[alcaldia].varValue for alcaldia in lista_alcaldias)
    print(f"{dependencia}: {asignadas} ambulancias asignadas de {ambulancias_disponibles[dependencia]} disponibles")


Estado de la solución: 1
Asignación de ambulancias por alcaldía:
Gustavo A Madero: 2.0 ambulancias
Azcapotzalco: 2.0 ambulancias
Cuauhtémoc: 2.0 ambulancias
Cuajimalpa: 2.0 ambulancias
Álvaro Obregón: 2.0 ambulancias
Coyoacán: 2.0 ambulancias
Magdalena Contreras: 2.0 ambulancias
Tlalpan: 2.0 ambulancias
Xochimilco: 2.0 ambulancias
Milpa Alta: 2.0 ambulancias
Tláhuac: 2.0 ambulancias
Iztacalco: 1.0 ambulancias
Iztapalapa: 2.0 ambulancias
Venustiano Carranza: 2.0 ambulancias
Benito Juárez: 1.0 ambulancias
Miguel Hidalgo: 2.0 ambulancias
ERUM: 20.0 ambulancias asignadas de 25 disponibles
CRUM: 8.0 ambulancias asignadas de 8 disponibles
Cruz Roja: 2.0 ambulancias asignadas de 4 disponibles


In [10]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum

# Datos
numberofSupply = 4  # Número de estaciones candidatas
numberofDemand = 16  # Número de nodos de demanda
p = 3  # Número total de estaciones activas permitidas
C = 50  # Capacidad máxima de cada estación
dem = [10, 20, 15, 25, 10, 30, 20, 40, 15, 35, 25, 30, 10, 20, 15, 25]  # Demanda de cada nodo
T = [  # Matriz de tiempos (estación -> nodo)
    [5, 10, 15, 20, 25, 30, 10, 15, 10, 5, 10, 25, 15, 20, 10, 25],
    [10, 15, 20, 25, 30, 5, 15, 20, 15, 10, 20, 5, 10, 15, 25, 20],
    [20, 25, 5, 10, 15, 20, 25, 30, 5, 10, 15, 20, 25, 30, 10, 15],
    [30, 5, 10, 15, 20, 25, 5, 10, 15, 20, 25, 30, 5, 10, 15, 20],
]

# Crear modelo
model = LpProblem("Minimizar_Tiempo_Transporte", LpMinimize)

# Variables de decisión
x = [LpVariable(f"x_{j}", cat="Binary") for j in range(numberofSupply)]  # Estaciones activas
y = [[LpVariable(f"y_{i}_{j}", cat="Binary") for j in range(numberofSupply)] for i in range(numberofDemand)]  # Asignación

# Función objetivo: minimizar el tiempo total de transporte ponderado por la demanda
model += lpSum(dem[i] * y[i][j] * T[j][i] for i in range(numberofDemand) for j in range(numberofSupply)), "Minimizar_Tiempo"

# Restricción: una estación debe estar activa para asignar demanda
for i in range(numberofDemand):
    for j in range(numberofSupply):
        model += y[i][j] <= x[j], f"Estacion_Activa_{i}_{j}"

# Restricción: exactamente p estaciones activas
model += lpSum(x[j] for j in range(numberofSupply)) == p, "Numero_Estaciones_Activas"

# Restricción: cada nodo de demanda debe ser cubierto por al menos una estación
for i in range(numberofDemand):
    model += lpSum(y[i][j] for j in range(numberofSupply)) == 1, f"Cobertura_Demanda_{i}"

# Restricción: capacidad máxima de cada estación
for j in range(numberofSupply):
    model += lpSum(y[i][j] * dem[i] for i in range(numberofDemand)) <= C, f"Capacidad_Estacion_{j}"

# Resolver modelo
model.solve()

# Mostrar resultados
print("Estado de la solución:", model.status)
print("Estaciones activadas:")
for j in range(numberofSupply):
    print(f"Estación {j + 1}: {'Activa' if x[j].varValue == 1 else 'Inactiva'}")

print("Asignación de nodos:")
for i in range(numberofDemand):
    for j in range(numberofSupply):
        if y[i][j].varValue == 1:
            print(f"Nodo {i + 1} asignado a estación {j + 1}")

# Calcular tiempo total mínimo
tiempo_total = sum(dem[i] * y[i][j].varValue * T[j][i] for i in range(numberofDemand) for j in range(numberofSupply))
print("Tiempo total de transporte mínimo:", tiempo_total)


Estado de la solución: -1
Estaciones activadas:
Estación 1: Activa
Estación 2: Activa
Estación 3: Inactiva
Estación 4: Inactiva
Asignación de nodos:
Nodo 2 asignado a estación 4
Nodo 3 asignado a estación 3
Nodo 4 asignado a estación 3
Nodo 6 asignado a estación 2
Nodo 7 asignado a estación 4
Nodo 10 asignado a estación 3
Nodo 11 asignado a estación 3
Nodo 14 asignado a estación 4
Nodo 16 asignado a estación 3
Tiempo total de transporte mínimo: 2924.9999985


In [18]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary, value

# Parámetros
number_of_supply = 5  # Número de estaciones(ambulancias)
number_of_demand = 8  # Número de nodos (alcaldias)
p = 3  # Número de estaciones a emplear
C = 20  # Capacidad máxima de las estaciones

# Datos ficticios de demanda y tiempos
dem = [5, 8, 3, 7, 6, 4, 9, 10]  # Demanda de cada nodo
T = [
    [2, 4, 5, 3, 6, 8, 7, 9],
    [3, 5, 4, 2, 7, 6, 9, 8],
    [5, 3, 2, 4, 8, 7, 6, 9],
    [6, 8, 7, 5, 3, 2, 4, 9],
    [9, 7, 6, 8, 4, 3, 5, 2],
]  # Tiempos entre estaciones y nodos

# Crear el modelo
model = LpProblem("Optimización de estaciones de ambulancia", LpMinimize)

# Variables de decisión
x = [LpVariable(f"x_{j}", cat=LpBinary) for j in range(number_of_supply)]  # Activación de estaciones
y = [[LpVariable(f"y_{i}_{j}", cat=LpBinary) for j in range(number_of_supply)] for i in range(number_of_demand)]

# Función objetivo: minimizar el tiempo total ponderado
model += lpSum(dem[i] * y[i][j] * T[j][i] for i in range(number_of_demand) for j in range(number_of_supply))

# Restricciones

# Solo se asigna demanda a una estación activa
for i in range(number_of_demand):
    for j in range(number_of_supply):
        model += y[i][j] <= x[j], f"Activación_{i}_{j}"

# Número de estaciones a emplear
model += lpSum(x[j] for j in range(number_of_supply)) == p, "Número_de_estaciones"

# Cada nodo debe estar asignado al menos a una estación
for i in range(number_of_demand):
    model += lpSum(y[i][j] for j in range(number_of_supply)) >= 1, f"Asignación_nodo_{i}"

# Capacidad máxima de las estaciones
for j in range(number_of_supply):
    model += lpSum(y[i][j] * dem[i] for i in range(number_of_demand)) <= C, f"Capacidad_estación_{j}"

# Resolver el modelo
model.solve()

# Resultados
print(f"Estado de la solución: {model.status}")
print("Estaciones activadas:")
for j in range(number_of_supply):
    print(f"Estación {j+1}: {'Activa' if value(x[j]) > 0.5 else 'Inactiva'}")

print("\nAsignación de nodos:")
for i in range(number_of_demand):
    for j in range(number_of_supply):
        if value(y[i][j]) > 0.5:
            print(f" Nodo {i+1} asignado a estación {j}")

# Calcular y mostrar tiempos totales por estación y general
station_times = [0] * number_of_supply
total_time = 0

for i in range(number_of_demand):
    for j in range(number_of_supply):
        if value(y[i][j]) > 0.5:  # Si el nodo i está asignado a la estación j
            contribution = dem[i] * T[j][i]  # Contribución del nodo i a la estación j
            station_times[j] += contribution
            total_time += contribution
print("\n")
# Mostrar resultados
for j in range(number_of_supply):
    print(f"Tiempo total para la estación {j+1}: {station_times[j]:.2f}")

print(f"\nTiempo total general: {total_time:.2f}")



Estado de la solución: 1
Estaciones activadas:
Estación 1: Activa
Estación 2: Inactiva
Estación 3: Inactiva
Estación 4: Activa
Estación 5: Activa

Asignación de nodos:
 Nodo 1 asignado a estación 0
 Nodo 2 asignado a estación 0
 Nodo 3 asignado a estación 4
 Nodo 4 asignado a estación 0
 Nodo 5 asignado a estación 3
 Nodo 6 asignado a estación 3
 Nodo 7 asignado a estación 3
 Nodo 8 asignado a estación 4


Tiempo total para la estación 1: 63.00
Tiempo total para la estación 2: 0.00
Tiempo total para la estación 3: 0.00
Tiempo total para la estación 4: 62.00
Tiempo total para la estación 5: 38.00

Tiempo total general: 163.00


In [19]:
from geopy.distance import geodesic

# Coordenadas de las estaciones y los nodos
estaciones = [(19.494945028482274, -99.11974959687133)]  # Coordenadas (lat, lon)
nodos = [(19.68802389752674, -99.05744634831647)]

# Calcular la matriz de tiempos (en este caso, usando distancia)
T = []
for estacion in estaciones:
    fila = []
    for nodo in nodos:
        fila.append(geodesic(estacion, nodo).kilometers)  # Distancia en km
    T.append(fila)

print(T)


[[22.350843303174685]]
